In [5]:
import matplotlib
import boto3
from botocore.exceptions import ClientError
from netCDF4 import Dataset
import datetime as dt
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import xmltodict
import yaml
import os
import subprocess
from datetime import date
from datetime import datetime
import logging
logger = logging.getLogger()
import os
import netCDF4
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point
from geopy.distance import distance
from pathlib import Path
import shutil

In [6]:
logger.setLevel(level=logging.INFO)

# Open configuration file and read paramters

In [7]:
with open(r'config.yaml') as file:
    configuration =  yaml.load(file, Loader=yaml.FullLoader)

In [8]:
for config in configuration['configurations']:
    username = config['username']
    password = config['password']
    url = config['url']
    aoi = config['aoi']
    awss3bucket = config['awss3bucket']
    awskeyid = config['awskeyid']
    awskeypass = config['awskeypass']

# Functions

In [53]:
def get_satellite_swaths(satsensorsA, satsensorsB, configuration, start, period):


    dirpath = Path('output')
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)
    
    try:
        subprocess.call(['python', 'swathpredict.py', '--configuration', configuration, '--start', start, '--period', period])
        success = True
    except:
        success = False
    
    filesA = []
    filesB = []
    
    for sat in satsensorsA:
   
        filesA.extend([f for f in os.listdir('output') if sat in f and 'swath.geojson' in f])
    
    for sat in satsensorsB:
    
        filesB.extend([f for f in os.listdir('output') if sat in f and 'swath.geojson' in f])
    
    gpdlistA = []
    for file in filesA:
        df = gpd.read_file(Path('output', file))
        gpdlistA.append(df)
    gpdlistB = []
    for file in filesB:
        df = gpd.read_file(Path('output',file))
        gpdlistB.append(df)        
    return(pd.concat(gpdlistA),pd.concat(gpdlistB) )    

In [56]:
# TODO for each pairwise comparison find the common imaged are per time period from start time
gpd1, gpd2 = get_satellite_swaths(['TERRA', 'AQUA'], ['NOAA_20', 'SUOMI_NPP'], 's3vtconfig.yaml', '2020-02-21T00:00:00Z', '1440')
gpd1 = gpd1.unary_union
gpd2 = gpd2.unary_union
intersection = gpd1.intersection(gpd2)

In [5]:
def get_file_list(username, password, aoi, startrecord):
    # rows returned is limited to 100, add pagination but looking at number of records and incrementing by 100 each iteration
    try:
        subprocess.call(['wget','--no-check-certificate', '--user='+username, '--password='+password, '--output-document=filelist.txt', 'https://131.176.236.38/dhus/search?q=footprint:"Intersects('+aoi+')" AND platformname:Sentinel-3 AND producttype:SL_2_FRP___&rows=100&start='+startrecord+'&format=json'])
        success = True
    except:
        success = False
    return(success)

In [6]:
def filter(sensors):

    for sensordict in sensors:
        
        filter_string = ''
        count = 0
        
        for sensor in sensordict.keys():
            filter_string = filter_string+'(sensor=%27'+sensor+'%27%20AND%20(product=%27'
            product_count = 0
            for product in sensordict[sensor]:
                filter_string = filter_string+product+'%27'
                if product_count < (len(sensordict[sensor])-1):
                    filter_string = filter_string+'%20OR%20product=%27'
                else:
                    filter_string = filter_string+'))' 
                product_count = product_count + 1
            if count < (len(sensordict.keys())-1):        
                filter_string = filter_string+'%20OR%20'
            count = count+1

    return(filter_string)

In [7]:
def get_polygon_from_gml(gml_dict):
    listoftuples = []
    for i in list(gml_dict.split(" ")):
        pair = (float(i.split(',')[1]), float(i.split(',')[0]))
        listoftuples.append(pair)
    return(listoftuples)

In [8]:
def load_hotspots(filter_string, time_period, bbox, max_features, min_confidence, to_date):
    y_max = bbox[0]
    x_min = bbox[1]
    y_min = bbox[2]
    x_max = bbox[3]
    if to_date is None:
        
        to_date = dt.datetime.now()
    
    
    from_date = (to_date - dt.timedelta(days=time_period)).strftime('%Y-%m-%d')
    
    # trim datetime to enable WFS 
    to_date = to_date.strftime('%Y-%m-%d')
    logger.info(str(from_date)+' '+str(to_date))
    
    # TODO - sort out paging - looks like there is a limit to WFS requests number returned per query
    logger.info(f"https://hotspots.dea.ga.gov.au/geoserver/public/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=public:hotspots&outputFormat=application/json&CQL_FILTER=({filter_string})%20AND%20datetime%20%3E%20%27{from_date}%27%20AND%20datetime%20%3C%20%27{to_date}%27%20AND%20INTERSECTS(location,%20POLYGON(({y_max}%20{x_min},%20{y_max}%20{x_max},%20{y_min}%20{x_max},%20{y_min}%20{x_min},%20{y_max}%20{x_min})))&maxFeatures={max_features}&startIndex=0&sortBy=sensor%20A")
    url = f"https://hotspots.dea.ga.gov.au/geoserver/public/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=public:hotspots&outputFormat=application/json&CQL_FILTER=({filter_string})%20AND%20datetime%20%3E%20%27{from_date}%27%20AND%20datetime%20%3C%20%27{to_date}%27%20AND%20INTERSECTS(location,%20POLYGON(({y_max}%20{x_min},%20{y_max}%20{x_max},%20{y_min}%20{x_max},%20{y_min}%20{x_min},%20{y_max}%20{x_min})))&maxFeatures={max_features}&startIndex=0&sortBy=sensor%20A"
    
    hotspots_gdf = gpd.read_file(url)
    #logger.info(str(hotspots_gdf['stop_dt']))
    
    # TODO - improved None value handling  -currently just look at first and apply that to all
    if hotspots_gdf['confidence'][0] == None:
        logger.info('Skipping confidence filter as confidence not populated')
    else:

        # Filter by confidence
        hotspots_gdf = hotspots_gdf.loc[hotspots_gdf.confidence >= min_confidence]

    # Fix datetime
    if hotspots_gdf['start_dt'][0] == None:
        logger.info('Start date field is not populated')
        hotspots_gdf['datetime'] = pd.to_datetime(hotspots_gdf['datetime'])
    else:
        hotspots_gdf['datetime'] = pd.to_datetime(hotspots_gdf['start_dt'])

    # Extract required columns
    hotspots_gdf = hotspots_gdf.loc[:, [
            'datetime', 'latitude', 'longitude', 'confidence', 'geometry', 'product', 'satellite', 'sensor', 'power'
            ]]
    hotspots_gdf.sort_values('datetime', ascending=True, inplace=True)
    logger.info('Hotspots loaded successfully '+str(hotspots_gdf.geometry.total_bounds))

    return(hotspots_gdf)

In [9]:
def solar_day(utc, longitude):
    SECONDS_PER_DEGREE = 240
    offset_seconds = int(longitude * SECONDS_PER_DEGREE)
    offset = np.timedelta64(offset_seconds, 's')
    return (np.datetime64(utc) + offset).astype(datetime)

In [10]:
def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True).add_prefix('2_'),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

# Skip to "Load Hotspots to GeoPandas" if testing analytics

# Assess inventory against AWS bucket listing

In [ ]:
# Get bucket listing of image granules and hotspot files

s3 = boto3.resource('s3', aws_access_key_id=awskeyid,
                    aws_secret_access_key=awskeypass)

s3folderlist = []
s3geojsonlist = []
s3bucket = s3.Bucket('s3vtaustralia')

for bucket_object in s3bucket.objects.all():
    s3bucketobject = str(bucket_object.key).split("/")[2]
    if '.SEN3' in s3bucketobject:
        s3folderlist.append(s3bucketobject)
    if '.FRP.geojson' in s3bucketobject:
        s3geojsonlist.append(bucket_object.key)

print(len(s3geojsonlist), "S3 Hotspot files From ESA available in AWS S3") 

In [ ]:
print(len(set(s3folderlist)), "S3 Granules From ESA available in AWS S3") 

# Retrieve the file list inventory used to gather the S3 files

In [ ]:
# Get File List from server
# Run this if no local inventory exists
if not os.path.exists('s3vt_inventory.json'):
    startrecord = 0

    responselist = [] 

    # Determine number of records to retrieve
    get_file_list(username, password, aoi, str(startrecord))
    with open('filelist.txt') as results:
        for i in results: 
            response = json.loads(i)
            responselist.append(response)

    upperlimit = int(response['feed']['opensearch:totalResults'])
    upperlimit = 200
    # Get the full list of records

    while startrecord <= upperlimit:
        startrecord = startrecord+100

        get_file_list(username, password, aoi, str(startrecord)) 
        with open('filelist.txt') as results:
            for i in results: responselist.append(json.loads(i))

            # Dump the results to an inventory file
            with open('s3vt_inventory.json', 'w') as f:
                json.dump(responselist, f)

# From the inventory - generate a vector fooprint

In [ ]:
# Read inventory to geopandas - write to geojson       
        
with open('s3vt_inventory.json') as inventory:
    frames = []
    for p in inventory:
        pages = json.loads(p)
                
        for page in pages:
            for page in pages:
                try:
                    
                    for entry in page['feed']['entry']:

                        df = pd.DataFrame.from_dict(entry, orient='index')

                        polygon = get_polygon_from_gml(xmltodict.parse(entry['str'][2]['content'])['gml:Polygon']['gml:outerBoundaryIs']['gml:LinearRing']['gml:coordinates'])

                        df = df.transpose()
                        df['Coordinates'] = Polygon(polygon)
                        for d in entry['str']:
                            if d['name'] ==  'orbitdirection':
                                df['orbitdirection'] = d['content']
                            if d['name'] ==  'platformidentifier':
                                df['platformidentifier'] = d['content'] 
                            if d['name'] ==  'filename':
                                df['filename'] = d['content']
                            if d['name'] ==  'instrumentshortname':
                                df['instrumentshortname'] = d['content']
                            if d['name'] ==  'passnumber':
                                df['passnumber'] = d['content']        
                        s3vtdf = gpd.GeoDataFrame(df, geometry='Coordinates')

                        frames.append(s3vtdf)
                                
                except KeyError:
                        logger.info("KeyError exception for get_polygon_from_gml()")
                        
s3vtgpd = pd.concat(frames)

# Not sure why we need to index but do it anyway
s3vtgpd = s3vtgpd.reset_index(drop=True)
s3vtgpd['date'] = pd.to_datetime(s3vtgpd.summary.str.split(",", expand= True)[0].str.split(' ', expand=True)[1])
# Some fields are lists and geojson translation doesn't like it

s3vtgpd = s3vtgpd.drop(['link', 'int', 'str', 'summary'], axis=1)
s3vtgpd.to_file('s3vt_geometry.geojson', driver='GeoJSON')

# Set up for checking inventory against files on AWS

In [ ]:
dataframelength = len(s3vtgpd)
# Add field to enable monitoring
s3vtgpd['hotspot'] = 0
s3vtgpd['download'] = 0
s3vtgpd['s3bucket'] = 0

s3vthostpotsgpdlist = []

# Run inventory check

In [ ]:
# Check if folder already downloaded and flag in gpd
for i in range(dataframelength):
    if s3vtgpd.loc[i]['title']+'.SEN3' in set(s3folderlist):
        s3vtgpd.at[i, 'download'] = 1
    if s3vtgpd.loc[i]['title']+'.FRP.geojson' in set(s3folderlist):
        s3vtgpd.at[i, 'hotspot'] = 1
        s3vthostpotsgpdlist.append(s3hotspotsgpd)

# Sync GeoJSON to local machine

In [ ]:
# Copy all S3 geojson files locally and load to GPD
# TODO - fix the below subprocess as it doesn't seem to accept the exclude include parameters and syncs everything
#subprocess.call(['echo', 'aws', 's3', 'sync', 's3://s3vtaustralia/', '.', '--exclude', '\"*\"', '--include', '\"*.geojson\"', '--dryrun'])
# Going with this in the interim
!aws s3 sync s3://s3vtaustralia/ . --exclude "*" --include "*.geojson"

#for i in s3geojsonlist:
#    print(i)
#    subprocess.call(['aws', 's3', 'sync', 's3://s3vtaustralia/'+i, i, '--dryrun'])

# Filter hotspots based on limitations stated by UCL

In [ ]:
frames = []
for i in s3geojsonlist:
    df1 = gpd.read_file(i)
    if 'S3A' in i:
        df1['satellite'] = 'S3A'
    else:
        df1['satellite'] = 'S3B'
    df1['sensor'] = 'SLSTR'
    df2 = df1.query("FRP_MWIR>0")
    if len(df2) > 0:
        frames.append(df2)
        
s3vthotspots = pd.concat(frames)
#for i in s3geojsonlist:
#    s3vtdf = gpd.GeoDataFrame(df, geometry='Coordinates')

# Convert netcdf CF time to something pandas understands

In [ ]:
s3vthotspots['date'] = pd.to_datetime(netCDF4.num2date(s3vthotspots.time, units='microseconds since 2000-01-01T00:00:00Z', only_use_cftime_datetimes=False, only_use_python_datetimes=True))

# Add solar day column to enable group by function

In [ ]:
s3vthotspots['solar_day'] = s3vthotspots.apply(lambda row: solar_day(row.date, row.longitude), axis = 1)

# Write S3 Hotspots to GeoJSON

In [ ]:
s3vthotspots.to_file('s3vt_hotspots.geojson', driver='GeoJSON')

In [ ]:
bounds = list(s3vthotspots.geometry.total_bounds)
bbox = [bounds[3], bounds[0], bounds[1], bounds[2]]
bbox

In [ ]:
# Get time bounds for DEA Hotspots query
# TODO - AHI hotspots

maxdate = s3vthotspots.date.max().to_datetime64()
mindate = s3vthotspots.date.min().to_datetime64()
to_date = dt.datetime.strptime(str(s3vthotspots.date.max().to_datetime64()), '%Y-%m-%dT%H:%M:%S.%f000') # '2018-01-01T00:00:00.000Z'
time_period = int(str(np.timedelta64(maxdate - mindate, 'D')).split(' ')[0])

In [ ]:
for config in configuration['configurations']:
    
    hotspots_gdf = load_hotspots(filter(config['sensors']),
                                         time_period,
                                         bbox,
                                         config['max_features'], 
                                         config['min_confidence'],
                                         to_date)
hotspots_gdf['solar_day'] = hotspots_gdf.apply(lambda row: solar_day(row.datetime, row.longitude), axis = 1) 

In [ ]:
try:
    os.remove('DEAHotspots_hotspots.geojson')
except:
    logger.info('DEA Hotspots geojson does not exist or cannot be deleted')
    
hotspots_gdf.to_file('DEAHotspots_hotspots.geojson', driver='GeoJSON')

In [ ]:
hotspots_gdf.datetime.max()

In [ ]:
s3vthotspots.date.max()

In [ ]:
s3vthotspots.plot()

In [ ]:
hotspots_gdf.plot()

# Load Hotspots to GeoPandas

In [15]:
hotspots_gdf = gpd.read_file('DEAHotspots_hotspots.geojson')
hotspots_gdf['datetime'] = pd.to_datetime(hotspots_gdf['datetime'])
hotspots_gdf['solar_day'] = pd.to_datetime(hotspots_gdf['solar_day'])

In [ ]:
s3vthotspots = gpd.read_file('s3vt_hotspots.geojson')
s3vthotspots['datetime'] = pd.to_datetime(s3vthotspots['date'])
s3vthotspots['solar_day'] = pd.to_datetime(s3vthotspots['solar_day'])

In [ ]:
hotspot

# Merge to a single GeoPandas Datafame

In [13]:
# TODO

# Set products for comparison

In [14]:
products = [{'product': 'SRSS', 'sensor': 'MODIS', 'satellite': 'TERRA'},
 {'product': 'MOD14', 'sensor': 'MODIS', 'satellite': 'TERRA'},
 {'product': 'SRSS', 'sensor': 'MODIS', 'satellite': 'AQUA'},
 {'product': 'MOD14', 'sensor': 'MODIS', 'satellite': 'AQUA'},                        
 {'product': 'SRSS', 'sensor': 'AVHRR', 'satellite': 'NOAA 19'},
 {'product': 'GA', 'sensor': 'AVHRR', 'satellite': 'NOAA 19'},
 {'product': 'SRSS', 'sensor': 'VIIRS', 'satellite': 'NOAA 20'},
 {'product': 'AFMOD', 'sensor': 'VIIRS', 'satellite': 'NOAA 20'},
 {'product': 'EDR', 'sensor': 'VIIRS', 'satellite': 'NOAA 20'},
 {'product': 'SRSS', 'sensor': 'VIIRS', 'satellite': 'SUOMI NPP'},
 {'product': 'AFMOD', 'sensor': 'VIIRS', 'satellite': 'SUOMI NPP'},
 {'product': 'EDR', 'sensor': 'VIIRS', 'satellite': 'SUOMI NPP'}] 

# Index by solar time to enable groupby

In [15]:
hotspots_gdf.set_index(pd.DatetimeIndex(hotspots_gdf.solar_day.values))


,datetime,latitude,longitude,confidence,product,satellite,sensor,power,solar_day,geometry
2020-02-03 09:46:58,2020-02-02 23:51:16,-35.777,148.927,100,MOD14,TERRA,MODIS,123.6,2020-02-03 09:46:58,POINT (148.92700 -35.77700)
2020-02-03 09:46:28,2020-02-02 23:51:16,-35.615,148.801,66,MOD14,TERRA,MODIS,32.8,2020-02-03 09:46:28,POINT (148.80100 -35.61500)
2020-02-03 09:47:22,2020-02-02 23:51:16,-35.652,149.026,68,MOD14,TERRA,MODIS,23.8,2020-02-03 09:47:22,POINT (149.02600 -35.65200)
2020-02-03 09:46:27,2020-02-02 23:51:16,-35.624,148.798,73,MOD14,TERRA,MODIS,42.6,2020-02-03 09:46:27,POINT (148.79800 -35.62400)
2020-02-03 09:46:30,2020-02-02 23:51:16,-35.626,148.810,100,MOD14,TERRA,MODIS,197.2,2020-02-03 09:46:30,POINT (148.81000 -35.62600)
...,...,...,...,...,...,...,...,...,...,...
2020-04-22 09:27:58,2020-04-21 23:15:42,-25.980,153.070,50,SRSS,TERRA,MODIS,12.8,2020-04-22 09:27:58,POINT (153.07000 -25.98000)
2020-04-22 09:27:56,2020-04-21 23:15:42,-25.980,153.060,50,SRSS,TERRA,MODIS,12.8,2020-04-22 09:27:56,POINT (153.06000 -25.98000)
2020-04-22 09:27:54,2020-04-21 23:15:42,-25.980,153.050,50,SRSS,TERRA,MODIS,12.9,2020-04-22 09:27:54,POINT (153.05000 -25.98000)
2020-04-22 09:27:54,2020-04-21 23:15:42,-25.970,153.050,50,SRSS,TERRA,MODIS,12.9,2020-04-22 09:27:54,POINT (153.05000 -25.97000)


In [16]:
s3vthotspots.set_index(pd.DatetimeIndex(s3vthotspots.solar_day.values))

,F1_Fire_pixel_radiance,FRP_MWIR,FRP_SWIR,FRP_uncertainty_MWIR,FRP_uncertainty_SWIR,Glint_angle,IFOV_area,Radiance_window,S7_Fire_pixel_radiance,TCWV,...,time,transmittance_MWIR,transmittance_SWIR,used_channel,satellite,sensor,date,solar_day,geometry,datetime
2020-02-02 22:11:07,0.17,9.095869,9.68015,1.075820,14.428136,100.000987,1498243.500,0.16,0.45,15.562044,...,633960916768279,0.802744,1.619436,0,S3A,SLSTR,2020-02-02T12:15:16,2020-02-02 22:11:07,POINT (148.96626 -37.22027),2020-02-02 12:15:16
2020-02-02 22:10:17,0.26,2.933022,-1.00000,1.098716,0.000000,98.470242,1543194.000,0.21,0.30,15.498196,...,633960917969608,0.801471,-1.000000,0,S3A,SLSTR,2020-02-02T12:15:17,2020-02-02 22:10:17,POINT (148.75155 -37.25186),2020-02-02 12:15:17
2020-02-02 22:10:14,0.44,4.478996,-1.00000,1.449789,0.000000,98.394357,1545956.875,0.24,0.37,15.531377,...,633960917969690,0.801227,-1.000000,0,S3A,SLSTR,2020-02-02T12:15:17,2020-02-02 22:10:14,POINT (148.73992 -37.25800),2020-02-02 12:15:17
2020-02-02 22:11:10,0.16,4.994986,-1.00000,0.550582,0.000000,100.082946,1495717.750,0.15,0.31,15.538684,...,633960917068182,0.802935,-1.000000,0,S3A,SLSTR,2020-02-02T12:15:17,2020-02-02 22:11:10,POINT (148.97391 -37.20723),2020-02-02 12:15:17
2020-02-02 22:11:08,0.63,17.271609,-1.00000,1.759986,0.000000,100.006031,1498243.500,0.16,0.70,15.505946,...,633960917068264,0.803017,-1.000000,0,S3A,SLSTR,2020-02-02T12:15:17,2020-02-02 22:11:08,POINT (148.96274 -37.21304),2020-02-02 12:15:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-22 09:32:34,0.43,9.080075,-1.00000,0.968183,0.000000,66.657121,1156394.500,0.28,0.67,10.377250,...,640827905944053,0.842408,0.000000,0,S3B,SLSTR,2020-04-21T23:45:05,2020-04-22 09:32:34,POINT (146.87343 -36.68392),2020-04-21 23:45:05
2020-04-22 09:47:10,0.47,16.401955,-1.00000,1.955553,0.000000,52.845568,0.000,0.47,0.74,13.567919,...,640830231514760,0.819852,0.000000,1,S3A,SLSTR,2020-04-22T00:23:51,2020-04-22 09:47:10,POINT (140.83282 -34.09221),2020-04-22 00:23:51
2020-04-22 10:01:18,0.84,10.207374,-1.00000,2.661284,0.000000,34.458620,0.000,0.56,0.74,53.808964,...,640835697549905,0.666571,0.000000,1,S3A,SLSTR,2020-04-22T01:54:57,2020-04-22 10:01:18,POINT (121.59141 1.04187),2020-04-22 01:54:57
2020-04-22 10:13:33,0.76,15.103070,-1.00000,1.770896,0.000000,19.488487,0.000,0.49,0.73,48.303738,...,640835748877024,0.658864,0.000000,1,S3A,SLSTR,2020-04-22T01:55:48,2020-04-22 10:13:33,POINT (124.43958 -1.97301),2020-04-22 01:55:48


# Run comparison matrix

In [17]:
appended_dataframe = []

# For each DEA Hotspots solar day group
for name, group in hotspots_gdf.resample('D', on='solar_day'):
    #print(group.datetime.max() - group.datetime.min(), name)
    # TODO predict swaths for time interval
    # and each S3 Hotspots solar day group
    for s3name, s3group in s3vthotspots.resample('D', on='solar_day'):      
        # where the solar days are the same
        if (name == s3name):
            # On each unique DEA Hotspots solar group
            for product in products:
                # each product represents a unique sensor, product and satellite
               
                dea_hotspot_product = (group[(group['sensor'] == product['sensor']) &\
                                            (group['product'] == product['product']) &\
                                            (group['satellite'] == product['satellite'])]).reset_index()
                dea_hotspot_product['solar_date'] = name
                # If one of the dataframes going into the nearest hotspot function is empty this fails so try except
                
                try:
                    per_solarday_nearest_hotspots = ckdnearest(dea_hotspot_product , s3group.reset_index())
                    
                    appended_dataframe.append(per_solarday_nearest_hotspots)
                except:
                    logger.info('Skipping')
                # Clip to common intersection of predicted swaths
                
dea_s3_nearest_hotspot_gdf = pd.concat(appended_dataframe)               

# Add metres distance between two points

In [19]:
dea_s3_nearest_hotspot_gdf['dist_m'] = dea_s3_nearest_hotspot_gdf.apply(lambda row: distance((row.latitude, row.longitude),(row['2_latitude'], row['2_longitude'])).meters, axis = 1)

In [ ]:
#TODO polygon intersection (using satellite footprint tool and S3 geojson)
#TODO match available Landsat and Sentinel 2 datasets with hotspots
#TODO run fire detection on Landsat and Sentinel 2 - update attributes
#TODO rerun the nearest test with S3 as the primary, as well as DEA Hotspots.
#TODO add Himawari Hotspots from SRSS and WFABBA.
#TODO add persistent hotspots and compare frequency of detection

In [ ]:
# Questions to answer
# How to define coincidence?
# How to constrain results - based on confidence? and minimum allowable radius?
# Perhaps groupby time should be a moving window of two days or only look at matching hotspots first detected early in the day?

In [20]:
# Add product group attribute to DEA dataframe
# TODO roll this up to the nearest comparison to simplify product handling
dea_s3_nearest_hotspot_gdf['product_group'] = dea_s3_nearest_hotspot_gdf['satellite'].map(str) + '_' + dea_s3_nearest_hotspot_gdf['sensor'].map(str) + '_' + dea_s3_nearest_hotspot_gdf['product'].map(str)

## MODIS vs S3

In [ ]:
len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) & \
                               (dea_s3_nearest_hotspot_gdf['dist'] < 0.05) & \
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'MODIS')])

In [ ]:
len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) &\
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'MODIS')])

In [ ]:
dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['dist'] < 0.05) & (dea_s3_nearest_hotspot_gdf['sensor'] == 'MODIS')].plot()

## AVHRR vs S3

In [ ]:
len(dea_s3_nearest_hotspot_gdf[#(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) & \
    
                               (dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & \
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'AVHRR')])

In [ ]:
len(dea_s3_nearest_hotspot_gdf[#(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) & \
    
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'AVHRR')])

In [ ]:
dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & (dea_s3_nearest_hotspot_gdf['sensor'] == 'AVHRR')].plot()

In [ ]:
len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) & \
                               (dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & \
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'VIIRS')])

In [ ]:
len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 85) & \
                               (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0.90) & \
                               #(dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & \
                               (dea_s3_nearest_hotspot_gdf['sensor'] == 'VIIRS')])

In [ ]:
dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & (dea_s3_nearest_hotspot_gdf['sensor'] == 'VIIRS')].plot()

In [ ]:
dea_s3_nearest_hotspot_gdf.to_csv('dea_s3_nearest_hotspot_gdf.csv')

In [56]:
results = {}

In [57]:
for product_group in set(dea_s3_nearest_hotspot_gdf['product_group']):
    results[product_group+'_candidates'] = len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 0) & (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0) & (dea_s3_nearest_hotspot_gdf['product_group'] == product_group)])
    results[product_group+'_matches'] = len(dea_s3_nearest_hotspot_gdf[(dea_s3_nearest_hotspot_gdf['confidence'] > 0) & (dea_s3_nearest_hotspot_gdf['2_confidence'] > 0) & (dea_s3_nearest_hotspot_gdf['dist_m'] < 2000) & (dea_s3_nearest_hotspot_gdf['product_group'] == product_group)])
    results[product_group+'_percent_matched'] = (results[product_group+'_matches']/results[product_group+'_candidates'])*100

In [58]:
results

{'TERRA_MODIS_SRSS_candidates': 2055,
 'TERRA_MODIS_SRSS_matches': 1134,
 'TERRA_MODIS_SRSS_percent_matched': 55.18248175182482,
 'SUOMI NPP_VIIRS_SRSS_candidates': 3258,
 'SUOMI NPP_VIIRS_SRSS_matches': 493,
 'SUOMI NPP_VIIRS_SRSS_percent_matched': 15.131982811540823,
 'SUOMI NPP_VIIRS_EDR_candidates': 77,
 'SUOMI NPP_VIIRS_EDR_matches': 8,
 'SUOMI NPP_VIIRS_EDR_percent_matched': 10.38961038961039,
 'AQUA_MODIS_SRSS_candidates': 2488,
 'AQUA_MODIS_SRSS_matches': 677,
 'AQUA_MODIS_SRSS_percent_matched': 27.21061093247588,
 'SUOMI NPP_VIIRS_AFMOD_candidates': 6212,
 'SUOMI NPP_VIIRS_AFMOD_matches': 1556,
 'SUOMI NPP_VIIRS_AFMOD_percent_matched': 25.04829362524147,
 'NOAA 19_AVHRR_SRSS_candidates': 724,
 'NOAA 19_AVHRR_SRSS_matches': 280,
 'NOAA 19_AVHRR_SRSS_percent_matched': 38.67403314917127,
 'NOAA 20_VIIRS_SRSS_candidates': 2626,
 'NOAA 20_VIIRS_SRSS_matches': 390,
 'NOAA 20_VIIRS_SRSS_percent_matched': 14.85148514851485,
 'TERRA_MODIS_MOD14_candidates': 2530,
 'TERRA_MODIS_MOD14_ma

In [37]:
    results[i[product_group+'_matches']/results[product_group+'_candidates']